In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── 5.без клауз, включающих местоимения.txt
│   ├── 6.без слов функциональных.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   ├── Второй_жанр без клауз, включающих местоимения.txt
│   ├── Второй_жанр без слов функциональных.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъ

In [2]:
import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

In [3]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
#import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)


import json


def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        return obj.item()
    return obj

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False, default=convert_numpy)

        
def create_custom_config(model_name, model_type, dataset):
    """Crea configuración de entrenamiento adaptativa"""
    common_config = {
#         'dataset_name':dataset.get('name', ''),
        'model_name': model_name,
        'model_type': model_type,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5
    }

    # Configuración específica para GPT
#     if model_type == 'gpt':
#         return funciones.TrainingConfig(
#             **common_config,
#             max_length=52,
#             batch_size=64,
#             epochs=6,
#             learning_rate=1e-5
#         )

#         # Configuración por nombre de dataset
#         dataset_name = dataset.get('name', '')
#         if any(x in dataset_name for x in ['1', '2', '3', '4']):
#             return funciones.TrainingConfig(
#                 **common_config,
#                 max_length=60,
#                 batch_size=128,
#                 epochs=6,
#                 learning_rate=5e-5
#             )


    # Configuraciones basadas en frecuencia
    freq_threshold = dataset.get('freq_threshold')
    if freq_threshold in [100, 49, 29, 9, 5, 3]:
        configs = {
            100: (52, 16, 6, 2e-5),
            49: (60, 16, 6, 3e-5),
            29: (51, 16, 6, 3e-5),
            9: (45, 16, 6, 4e-5),
            5: (150, 16, 6, 5e-5),
            3: (150, 16, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[freq_threshold]
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por nombre de dataset
    dataset_name = dataset.get('name', '')
    for x in ['1', '2', '3', '4', '5', '6']:
        if x in dataset_name:
            configs = {
                '1': (60, 16, 6, 5e-5),
                '2': (60, 16, 6, 5e-5),
                '3': (60, 16, 6, 5e-5),
                '4': (60, 16, 6, 5e-5),
                '5': (60, 16, 6, 5e-5),
                '6': (60, 16, 6, 5e-5)
            }
            max_len, batch, epochs, lr = configs[x]
            return funciones.TrainingConfig(
                **common_config,
                max_length=max_len,
                batch_size=batch,
                epochs=epochs,
                learning_rate=lr
            )


    # Configuración por defecto
    return funciones.TrainingConfig(
        **common_config,
        max_length=60,
        batch_size=32,
        epochs=4,
        learning_rate=3e-5
    )

models = [
    {'model':'DeepPavlov/rubert-base-cased',
     'name':'DeepPavlov-rubert-base',
     'type': 'bert'}, # Modelo original (ruso)
    {'model':'bert-base-multilingual-cased',
     'name':'BERT multilingual',
     'type': 'bert'}, # BERT multilingüe
    {'model':'distilbert-base-multilingual-cased',
     'name':'distilbert-base-multilingual',
     'type': 'bert'}, # Versión ligera de BERT multilingüe
    {'model':'roberta-base',
     'name':'roberta-base',
     'type': 'bert'}, # RoBERTa (principalmente inglés)

    {'model':'xlnet-base-cased',
     'name':'XLNet Base',
     'type': 'bert'}, # XLNet (multilingüe efectivo)
    {'model':'xlm-roberta-base',
     'name':'XLM-RoBERTa Base',
     'type': 'bert'}, # XLM-RoBERTa (multilingüe)


    {'model': 'gpt2',
     'name': 'gpt2',
     'type': 'gpt'},  # GPT model
    {'model': 'facebook/opt-125m',
     'name': 'facebook/opt-125m',
     'type': 'gpt'},  # GPT model
    {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
     'name': 'rugpt3small',
     'type': 'gpt'}, # GPT model (ruso)
#         {
#         "model": "facebook/mbart-large-50",
#         "name": "mBART Large 50",
#         "type": "gpt"
#         }
    ]

datasets = [
    #deberia adicionar el set original?
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
        {
            'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
            'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
            'name': 'Изъяты лексемы с частотой выше 49',
            'type': 'freq',
            'freq': 49
        },
        {
            'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
            'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
            'name': 'Изъяты лексемы с частотой выше 29',
            'type': 'freq',
            'freq': 29
        },
        {
            'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
            'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
            'name': 'Изъяты лексемы с частотой выше 9',
            'type': 'freq',
            'freq': 9
        },
        {
            'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
            'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
            'name': 'Изъяты лексемы с частотой выше 5',
            'type': 'freq',
            'freq': 5
        },
        {
            'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
            'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
            'name': 'Изъяты лексемы с частотой выше 3',
            'type': 'freq',
            'freq': 3
        },

            {
            'path1': '../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
            'path2': '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt',
            'name': 'Без прилагательных первый-второй жанр',#litle correction
            'type': 'pos',
            'freq': None
            },
          {
            'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '1.Первый жанр исходная выборка',# este el el orignal
            'type': 'pos',
            'freq': None

        },
        {
            'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '2.Первый жанр без клауз, включающих наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '3.Первый жанр без клауз, включающих глаголы',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/5.без клауз, включающих местоимения.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '5.без клауз, включающих местоимения.txt',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/6.без слов функциональных.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '6.без слов функциональных.txt',
            'type': 'pos',
            'freq': None
        },
    
    ]



russian_templates = [
    "Этот литературный фрагмент принадлежит к жанру {}.",
    "Данный текст является примером жанра {}.",
    "Стилистические особенности указывают на жанр {}.",
    "Жанровая принадлежность этого текста - {}.",
    "Эксперты классифицируют этот текст как жанр {}.",
    "Это типичный пример жанра {} в русской литературе.",
    "По ключевым характеристикам это текст жанра {}.",
    "Данное произведение относится к направлению {}.",
    "Анализ содержания позволяет отнести текст к жанру {}.",
    "Этот отрывок характерен для жанра {}."
]

Классифицируй следующий текст как "определение" или "описание". Текст, который кратко и формально объясняет понятие, является "определением". Текст, который подробно и образно описывает объект или явление, является "описанием". Текст: {text}  
Ответ: {label}
    
def main():

    results = []
    for model in models:
        for dataset in datasets:
            config = create_custom_config(model['model'], model['type'], dataset)
            
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = funciones.train_and_evaluate_dataset(
                dataset['path1'],
                dataset['path2'],
                config,
                dataset['name'],
                dataset['type']
                
            )
            results.append(result) 
#             if model['type'] = 'gpt':
                
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
    
    save_results(results, 'resultados_Step_8.json') 
    
    
    

    
if __name__ == "__main__":
    main()

2025-04-23 18:51:06.725752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv
Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([195])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.15260525305617462
Epoch 2, Loss: 0.04537013787086363
Epoch 3, Loss: 0.010898670166541167
Epoch 4, Loss: 0.002245299813458351
Epoch 5, Loss: 0.08411089717669205
Epoch 6, Loss: 0.006139527890977829
all_preds shape: (195,)
all_labels shape: (195,)
all_probs shape: (195, 2)
Train labels sample: tensor([1, 0, 1, 0, 0]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([198])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.1516093977778736
Epoch 2, Loss: 0.027587906804573896
Epoch 3, Loss: 0.0316382534090533
Epoch 4, Loss: 0.029210287937463296
Epoch 5, Loss: 0.019230314090641007
E

Epoch 2, Loss: 0.07085742538964207
Epoch 3, Loss: 0.04996554768982936
Epoch 4, Loss: 0.0027789535064419563
Epoch 5, Loss: 0.0012219659126871689
Epoch 6, Loss: 0.0061402032733894885
all_preds shape: (211,)
all_labels shape: (211,)
all_probs shape: (211, 2)
Train labels sample: tensor([0, 0, 0, 1, 0]), Shape: torch.Size([869])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([209])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.20310765788467092
Epoch 2, Loss: 0.05638093565252017
Epoch 3, Loss: 0.009596149157732724
Epoch 4, Loss: 0.0012757855308750136
Epoch 5, Loss: 0.0005365629775703631
Epoch 6, Loss: 0.04999562518628822
all_preds shape: (209,)
all_labels shape: (209,)
all_probs shape: (209, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([869])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([203])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1,

Train labels sample: tensor([1, 1, 0, 0, 0]), Shape: torch.Size([90])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([739])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.45187195142110187
Epoch 2, Loss: 0.0923530130336682
Epoch 3, Loss: 0.06562221903974812
Epoch 4, Loss: 0.12892841904734573
Epoch 5, Loss: 0.005226854079713424
Epoch 6, Loss: 0.00703642494045198
all_preds shape: (739,)
all_labels shape: (739,)
all_probs shape: (739, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([90])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([718])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.4313325087229411
Epoch 2, Loss: 0.10302954353392124
Epoch 3, Loss: 0.024288286299755175
Epoch 4, Loss: 0.006094369649266203
Epoch 5, Loss: 0.0029655122198164463
Epoch 6, Loss: 0.00162203829192246
all_preds shape: (718,)
all_labels shape: (718,)
all_probs

Epoch 1, Loss: 0.12289655950259079
Epoch 2, Loss: 0.02773738152940165
Epoch 3, Loss: 0.001533306574194946
Epoch 4, Loss: 0.03812040504152802
Epoch 5, Loss: 0.0009426833114544438
Epoch 6, Loss: 0.00038603948143480177
all_preds shape: (174,)
all_labels shape: (174,)
all_probs shape: (174, 2)
Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([876])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([170])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.12213258830491792
Epoch 2, Loss: 0.011068819162689827
Epoch 3, Loss: 0.007977277514609424
Epoch 4, Loss: 0.015369299854765732
Epoch 5, Loss: 0.1411484064110978
Epoch 6, Loss: 0.04644605520350689
all_preds shape: (170,)
all_labels shape: (170,)
all_probs shape: (170, 2)
Train labels sample: tensor([0, 0, 0, 1, 0]), Shape: torch.Size([876])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([172])
##########################DeepPavlov/rubert-base-cased---b

...

Epoch 4, Loss: 0.07892738439742539
Epoch 5, Loss: 0.062379491112936857
Epoch 6, Loss: 0.02283562210036276
all_preds shape: (189,)
all_labels shape: (189,)
all_probs shape: (189, 2)
train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv
Train labels sample: tensor([1, 1, 0, 0, 1]), Shape: torch.Size([830])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([184])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.2519529815069221
Epoch 2, Loss: 0.07876384236442391
Epoch 3, Loss: 0.03640925845593301
Epoch 4, Loss: 0.011123203076959517
Epoch 5, Loss: 0.0010618444427002226
Epoch 6, Loss: 0.00032499776649895187
all_preds shape: (184,)
all_labels shape: (184,)
all_probs shape: (184, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([830])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([203])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gp

Train labels sample: tensor([0, 1, 0, 0, 0]), Shape: torch.Size([825])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([226])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.3070651833236647
Epoch 2, Loss: 0.07523917413009958
Epoch 3, Loss: 0.0623335456397259
Epoch 4, Loss: 0.07517447957154498
Epoch 5, Loss: 0.059964946082730494
Epoch 6, Loss: 0.08194735824005105
all_preds shape: (226,)
all_labels shape: (226,)
all_probs shape: (226, 2)
Train labels sample: tensor([0, 1, 1, 0, 0]), Shape: torch.Size([825])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([218])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.2534935151054994
Epoch 2, Loss: 0.07085079572481426
Epoch 3, Loss: 0.029526322146571495
Epoch 4, Loss: 0.080867921616328
Epoch 5, Loss: 0.3642029954991393
Epoch 6, Loss: 0.12782829154121617
all_preds shape: (218,)
all_labels shape: (218,

##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.5390973749260107
Epoch 2, Loss: 0.20910155835251013
Epoch 3, Loss: 0.05043032253161073
Epoch 4, Loss: 0.014250618871301413
Epoch 5, Loss: 0.0010150938808995609
Epoch 6, Loss: 0.0015339121940390517
all_preds shape: (689,)
all_labels shape: (689,)
all_probs shape: (689, 2)
Train labels sample: tensor([0, 0, 1, 0, 0]), Shape: torch.Size([88])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([717])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.4879672701160113
Epoch 2, Loss: 0.1624983251094818
Epoch 3, Loss: 0.03805757143224279
Epoch 4, Loss: 0.01939927627487729
Epoch 5, Loss: 0.001127897048718296
Epoch 6, Loss: 0.0005483151156416474
all_preds shape: (717,)
all_labels shape: (717,)
all_probs shape: (717, 2)
Train labels sample: tensor([1, 0, 0, 0, 0]), Shape: torch.Size([88])
Test labels sample: tensor([0, 0, 0

Epoch 3, Loss: 0.03621752520235465
Epoch 4, Loss: 0.021917535391265928
Epoch 5, Loss: 0.026037222176059745
Epoch 6, Loss: 0.020089760474469906
all_preds shape: (167,)
all_labels shape: (167,)
all_probs shape: (167, 2)
train_2.Первый жанр без клауз, включающих наречия.csv
test_2.Первый жанр без клауз, включающих наречия.csv
Train labels sample: tensor([0, 0, 0, 1, 1]), Shape: torch.Size([840])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([182])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.14114807347573763
Epoch 2, Loss: 0.04471643440774642
Epoch 3, Loss: 0.005174568980674969
Epoch 4, Loss: 0.032632017809835406
Epoch 5, Loss: 0.016764722270454085
Epoch 6, Loss: 0.004841218141732626
all_preds shape: (182,)
all_labels shape: (182,)
all_probs shape: (182, 2)
Train labels sample: tensor([1, 1, 0, 1, 0]), Shape: torch.Size([840])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([177])
##############

Epoch 6, Loss: 0.0011468363828347112
all_preds shape: (238,)
all_labels shape: (238,)
all_probs shape: (238, 2)
Train labels sample: tensor([1, 1, 1, 1, 1]), Shape: torch.Size([768])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([250])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.28196541920866974
Epoch 2, Loss: 0.055791372161062704
Epoch 3, Loss: 0.01662124648388878
Epoch 4, Loss: 0.006277429808316508
Epoch 5, Loss: 0.056863441041324826
Epoch 6, Loss: 0.02573689605196705
all_preds shape: (250,)
all_labels shape: (250,)
all_probs shape: (250, 2)
Train labels sample: tensor([0, 1, 1, 1, 0]), Shape: torch.Size([768])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([245])
##########################sberbank-ai/rugpt3small_based_on_gpt2---gpt####################
Epoch 1, Loss: 0.1880098757270995
Epoch 2, Loss: 0.042428759539082726
Epoch 3, Loss: 0.021698317178864574
Epoch 4, Loss: 0.004424026111090

In [4]:
#batch_size=128

In [5]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)